Importing required libraries

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd gdrive/MyDrive/Proj

/content/gdrive/MyDrive/Proj


In [3]:
import os
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
from tqdm import tqdm
import copy
import time

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torchvision.transforms as transforms
from torchvision import models
from torch import optim, nn
import torch.nn.functional as F

In [4]:
device = 'cpu'

if torch.cuda.is_available() :
    device = 'cuda'

Creating dataset

In [5]:
class ImageDataset(Dataset) :

    def __init__(self, transform) :
        self.root_path = 'PACS/kfold/'

        # Listing the domains
        self.domains = os.listdir(self.root_path)

        # Listing the classes 
        self.classes = os.listdir(self.root_path+'cartoon')

        # Transformations
        self.transforms = transform

        self.images = []
        self.domains_y = []
        self.classes_y = []

        for i_dom, domain in enumerate(self.domains) :
            for i_cla, cla in enumerate(self.classes) :
                for image in os.listdir(self.root_path+domain+'/'+cla) :
                    # Finding image path
                    image_path = self.root_path+domain+'/'+cla+'/'+image
                    img = cv2.imread(image_path)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    self.images.append(img)

                    # One hot encoding domain
                    domainVector = np.zeros(5)
                    domainVector[-1] = 1
                    domainVector[i_dom] = 1
                    self.domains_y.append(domainVector)

                    # One hot encoding class
                    classVector = np.zeros(7)
                    classVector[i_cla] = 1
                    self.classes_y.append(classVector)

        self.images = np.array(self.images)
        self.domains_y = np.array(self.domains_y)
        self.classes_y = np.array(self.classes_y)

        self.domains_y = torch.Tensor(self.domains_y)
        self.classes_y = torch.Tensor(self.classes_y)

    def __getitem__(self, index) :

        return self.transforms(self.images[index].astype('float')/255), self.domains_y[index], self.classes_y[index]

    def __len__(self) :
        return len(self.images)

Defining transforms

In [6]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [7]:
dataset = ImageDataset(transform=transform)

In [8]:
# Train and test split
train_dataset, val_dataset, test_dataset = random_split(dataset, [6000, 1000, 2991])

In [9]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=True)

In [10]:
dataloaders = {
    'train' : train_dataloader,
    'val' : val_dataloader,
    'test' : test_dataloader
}

dataset_sizes = {
    'train' : 6000,
    'val': 1000,
    'test' : 2991
}

In [11]:
def getWeightsAndBiasesCNN(in_dim, out_dim, kernel_size, no_domains) :

    weights = torch.zeros(out_dim, in_dim, kernel_size, kernel_size, no_domains+1)
    biases = torch.zeros(out_dim, no_domains+1)

    for i in range(no_domains+1) :
        conv_layer = nn.Conv2d(in_dim, out_dim, kernel_size)

        weights[:, :, :, :, i] = conv_layer.state_dict()['weight']
        biases[:, i] = conv_layer.state_dict()['bias']
    
    return weights, biases

In [12]:
def getWeightsAndBiasesDense(in_size, out_size, no_domains) :

    weight_mat = torch.zeros(out_size, in_size, no_domains+1)
    bias_mat = torch.zeros(out_size, no_domains+1)

    for i in range(no_domains+1) :
        linMod = nn.Linear(in_size, out_size)

        weight = linMod.weight.detach()
        bias = linMod.bias.detach()

        if not(i == no_domains) :
          weight_mat[:, :, i] = weight/no_domains
          bias_mat[:, i] = bias/no_domains

    return weight_mat, bias_mat

In [13]:
# Initializing the weights and biases

weight1, bias1 = getWeightsAndBiasesCNN(3, 96, 11, 4)

# Maxpool here

weight2, bias2 = getWeightsAndBiasesCNN(96, 256, 5, 4)

# Maxpool here

weight3, bias3 = getWeightsAndBiasesCNN(256, 384, 3, 4)
weight4, bias4 = getWeightsAndBiasesCNN(384, 384, 3, 4)
weight5, bias5 = getWeightsAndBiasesCNN(384, 256, 3, 4)

# Maxpool here
# Dropout here

weight6, bias6 = getWeightsAndBiasesDense(256*6*6, 4096, 4)

# Dropout here

weight7, bias7 = getWeightsAndBiasesDense(4096, 4096, 4)
weight8, bias8 = getWeightsAndBiasesDense(4096, 7, 4)

weight1 = weight1.to(device)
weight2 = weight2.to(device)
weight3 = weight3.to(device)
weight4 = weight4.to(device)
weight5 = weight5.to(device)
weight6 = weight6.to(device)
weight7 = weight7.to(device)
weight8 = weight8.to(device)

bias1 = bias1.to(device)
bias2 = bias2.to(device)
bias3 = bias3.to(device)
bias4 = bias4.to(device)
bias5 = bias5.to(device)
bias6 = bias6.to(device)
bias7 = bias7.to(device)
bias8 = bias8.to(device)

weight1.requires_grad = True
weight2.requires_grad = True
weight3.requires_grad = True
weight4.requires_grad = True
weight5.requires_grad = True
weight6.requires_grad = True
weight7.requires_grad = True
weight8.requires_grad = True

bias1.requires_grad = True
bias2.requires_grad = True
bias3.requires_grad = True
bias4.requires_grad = True
bias5.requires_grad = True
bias6.requires_grad = True
bias7.requires_grad = True
bias8.requires_grad = True

In [14]:
def CNNModelUndoBias(X, dom_vec, isTraining) :

    # Using AlexNet architecture here

    # Calculating the current weights and biases by taking inner product
    weight1_curr = torch.inner(weight1, dom_vec)
    weight2_curr = torch.inner(weight2, dom_vec)
    weight3_curr = torch.inner(weight3, dom_vec)
    weight4_curr = torch.inner(weight4, dom_vec)
    weight5_curr = torch.inner(weight5, dom_vec)
    weight6_curr = torch.inner(weight6, dom_vec)
    weight7_curr = torch.inner(weight7, dom_vec)
    weight8_curr = torch.inner(weight8, dom_vec)

    bias1_curr = torch.inner(bias1, dom_vec)
    bias2_curr = torch.inner(bias2, dom_vec)
    bias3_curr = torch.inner(bias3, dom_vec)
    bias4_curr = torch.inner(bias4, dom_vec)
    bias5_curr = torch.inner(bias5, dom_vec)
    bias6_curr = torch.inner(bias6, dom_vec)
    bias7_curr = torch.inner(bias7, dom_vec)
    bias8_curr = torch.inner(bias8, dom_vec)

    out = X.reshape(-1, 3, 227, 227)

    # Convolutional Layers

    out = F.conv2d(out, weight1_curr, bias1_curr, 4)
    out = F.relu(out)

    out = F.max_pool2d(out, 3, 2)

    out = F.conv2d(out, weight2_curr, bias2_curr, 1, 2)
    out = F.relu(out)

    out = F.max_pool2d(out, 3, 2)

    out = F.conv2d(out, weight3_curr, bias3_curr, 1, 1)
    out = F.relu(out)

    out = F.conv2d(out, weight4_curr, bias4_curr, 1, 1)
    out = F.relu(out)

    out = F.conv2d(out, weight5_curr, bias5_curr, 1, 1)
    out = F.relu(out)

    out = F.max_pool2d(out, 3, 2)

    out = F.dropout2d(out, 0.5, isTraining)

    # Fully connected layers

    out = out.reshape(1, -1)

    out = F.linear(out, weight6_curr, bias6_curr)
    out = F.relu(out)
    out = F.dropout(out, 0.5, isTraining)
    out = F.linear(out, weight7_curr, bias7_curr)
    out = F.relu(out)
    out = F.linear(out, weight8_curr, bias8_curr)
    out = F.softmax(out, 1)

    return out

In [15]:
epochs = 10
lr = 0.001

# Writing the training loop
for e in range(epochs) :

    running_loss = 0.0
    running_correct = 0
    
    for img, dom, cla in tqdm(train_dataset) :

        img = img.to(device)
        dom = dom.to(device)
        cla = cla.to(device)
        
        # Finding prediction
        pred = CNNModelUndoBias(img.float(), dom.float(), True)

        # Calculating loss
        loss = F.cross_entropy(pred, cla.reshape(-1, 7))

        running_loss += loss

        pred_class = torch.argmax(pred)
        actual_class = torch.argmax(cla)

        if(pred_class == actual_class) :
            running_correct += 1

        loss.backward()

        # Gradient descent
        with torch.no_grad():
            
            weight1 -= weight1.grad * lr
            weight2 -= weight2.grad * lr
            weight3 -= weight3.grad * lr
            weight4 -= weight4.grad * lr
            weight5 -= weight5.grad * lr
            weight6 -= weight6.grad * lr
            weight7 -= weight7.grad * lr
            weight8 -= weight8.grad * lr

            bias1 -= bias1.grad * lr
            bias2 -= bias2.grad * lr
            bias3 -= bias3.grad * lr
            bias4 -= bias4.grad * lr
            bias5 -= bias5.grad * lr
            bias6 -= bias6.grad * lr
            bias7 -= bias7.grad * lr
            bias8 -= bias8.grad * lr

            weight1.grad.zero_()
            weight2.grad.zero_()
            weight3.grad.zero_()
            weight4.grad.zero_()
            weight5.grad.zero_()
            weight6.grad.zero_()
            weight7.grad.zero_()
            weight8.grad.zero_()

            bias1.grad.zero_()
            bias2.grad.zero_()
            bias3.grad.zero_()
            bias4.grad.zero_()
            bias5.grad.zero_()
            bias6.grad.zero_()
            bias7.grad.zero_()
            bias8.grad.zero_()
        
    print(f'Epoch {e}/{epochs-1} | Loss : {running_loss/len(train_dataset)} | Accuracy : {running_correct/len(train_dataset)}')



100%|██████████| 6000/6000 [07:37<00:00, 13.12it/s]


Epoch 0/9 | Loss : 1.944976568222046 | Accuracy : 0.17916666666666667


100%|██████████| 6000/6000 [07:38<00:00, 13.10it/s]


Epoch 1/9 | Loss : 1.940592646598816 | Accuracy : 0.18483333333333332


100%|██████████| 6000/6000 [07:38<00:00, 13.08it/s]


...

Epoch 7/9 | Loss : 1.8733018636703491 | Accuracy : 0.25516666666666665


100%|██████████| 6000/6000 [07:38<00:00, 13.08it/s]


Epoch 8/9 | Loss : 1.8437268733978271 | Accuracy : 0.27416666666666667


100%|██████████| 6000/6000 [07:38<00:00, 13.09it/s]


Epoch 9/9 | Loss : 1.815737247467041 | Accuracy : 0.29833333333333334


In [16]:
# Now extracting the domain agnostic model
def domainAgnosticModel(X) :

  # Calculating the current weights and biases by taking inner product
  weight1_curr = weight1[:, :, :, :, -1]
  weight2_curr = weight2[:, :, :, :, -1]
  weight3_curr = weight3[:, :, :, :, -1]
  weight4_curr = weight4[:, :, :, :, -1]
  weight5_curr = weight5[:, :, :, :, -1]
  weight6_curr = weight6[:, :, -1]
  weight7_curr = weight7[:, :, -1]
  weight8_curr = weight8[:, :, -1]

  bias1_curr = bias1[:, -1]
  bias2_curr = bias2[:, -1]
  bias3_curr = bias3[:, -1]
  bias4_curr = bias4[:, -1]
  bias5_curr = bias5[:, -1]
  bias6_curr = bias6[:, -1]
  bias7_curr = bias7[:, -1]
  bias8_curr = bias8[:, -1]

  out = X.reshape(-1, 3, 227, 227)

    # Convolutional Layers

  out = F.conv2d(out, weight1_curr, bias1_curr, 4)
  out = F.relu(out)

  out = F.max_pool2d(out, 3, 2)

  out = F.conv2d(out, weight2_curr, bias2_curr, 1, 2)
  out = F.relu(out)

  out = F.max_pool2d(out, 3, 2)

  out = F.conv2d(out, weight3_curr, bias3_curr, 1, 1)
  out = F.relu(out)

  out = F.conv2d(out, weight4_curr, bias4_curr, 1, 1)
  out = F.relu(out)

  out = F.conv2d(out, weight5_curr, bias5_curr, 1, 1)
  out = F.relu(out)

  out = F.max_pool2d(out, 3, 2)

  out = F.dropout2d(out, 0.5, False)

  # Fully connected layers
  batch_size = 4

  out = out.reshape(batch_size, -1)

  out = F.linear(out, weight6_curr, bias6_curr)
  out = F.relu(out)
  out = F.dropout(out, 0.5, False)
  out = F.linear(out, weight7_curr, bias7_curr)
  out = F.relu(out)
  out = F.linear(out, weight8_curr, bias8_curr)
  out = F.softmax(out, 1)

  return out

In [17]:
# Testing loop
running_correct = 0
total = 0

for img, dom, cla in tqdm(train_dataloader) :
  img = img.to(device)
  dom = dom.to(device)
  cla = cla.to(device)

  pred = domainAgnosticModel(img.float())

  _, pred_list = torch.max(pred, 1)
  pred_list = pred_list.to(device)

  _, cla_list = torch.max(cla, 1)
  cla_list = cla_list.to(device)

  running_correct += torch.sum(pred_list == cla_list)

print(f'Accuracy of the test dataset : {running_correct/len(test_dataset)}')

100%|██████████| 1500/1500 [00:25<00:00, 59.92it/s]

Accuracy of the test dataset : 0.3095954358577728
